In [1]:
### 웹브라우저 오픈을 위한 라이브러리
# - 브라우저 제어
# - 설치해야 합니다. : pip install selenium
from selenium import webdriver

### 데이터 처리
import pandas as pd

### 동적 웹페이지 처리 라이브러리
# - 동적 웹페이지 처리처럼 html.parser 처리를 하지 않아도,
#   라이브러리가 알아서 내부적으로 처리해줌
from selenium.webdriver.common.by import By

### 시간 라이브러리
# - 브라우저의 로딩시간이 길어지는 경우에는
#  -> 프로그램 처리 시간도 늦춰주어야 합니다.
#  -> time.sleep(초)를 이용해서 늦춰주면 됩니다.
import time

from web_control import WebControl

In [ ]:
web_control = WebControl()
driver = web_control.getDriver("https://www.megabox.co.kr/movie")

In [ ]:
movie_info_list = []
web_control = WebControl()
driver = web_control.getDriver("https://www.megabox.co.kr/movie")

time.sleep(1)

for i in range(10):
    ### 영화목록 1개 추출하기, 클릭해서 정보 페이지로 전환하기
    web_control.click_movie_detail(movie_index=i)
    
    ### 페이지 전환 이후 전환된 페이지를 window(브라우저)에 인식 시키기
    web_control.setWindowPage()
    
    
    ### 영화 상세 정보 추출하기
    title_path = "p.title"
    # 영화제목이 있는 태그 정보 추출하기
    title_element = driver.find_element(By.CSS_SELECTOR, title_path)
    # 추출한 태그에서 영화제목 text 추출하여 출력하기
    title = title_element.text
    
    ### [실관람평] 추출하기
    score_path = "#mainMegaScore > p > em"
    # 실관람평 태그 추출하기
    score_element = driver.find_element(By.CSS_SELECTOR, score_path)
    # - 실관람평 텍스트 추출하기
    if score_element:
        # 관람평이 있을 경우, 해당 인덱스의 텍스트 추출
        score = score_element.text
    else:
        # 관람평이 없을 경우 0 처리
        score = 0
    
    ### [예매순위] 및 [예매율] 추출하기
    temp_path = "#contents > div.movie-detail-page > div.movie-detail-cont > div.info > div.rate > p.cont"
    temp_element = driver.find_element(By.CSS_SELECTOR, temp_path)
    if temp_element:
        temp_text = temp_element.text.split(" ")
        rate = temp_text[1][1:][:-2]
        ranking = temp_text[0][:-1]

    else:
        rate = 0
        ranking = 0
    
    ### [누적관객수] 추출하기
    audience_path = "div.info > div.audience > p > em"
    audience_element = driver.find_element(By.CSS_SELECTOR, audience_path)
    
    if audience_element:
        audience = audience_element.text.replace(',','')
    else:
        audientce = 0
    
    # print(f"영화제목 : {title}")
    # print(f"실관람평 : {score}")
    # print(f"예매순위 : {ranking}위")
    # print(f"예매율 : {rate}%")
    # print(f"누적관객수 : {audience}명")
    
    # 이전 페이지로 돌아감
    web_control.go_back()

    ### 데이터리스트에 변수 담기
    movie_info_list.append({"영화제목":title,
                            "실관람평":score,
                            "예매순위":ranking,
                            "예매율":rate,
                            "누적관객수":audience})

    
### 데이터프레임에 저장하기
movie_info = pd.DataFrame(movie_info_list)

web_control.shutDriver()

In [ ]:
movie_info

In [ ]:
movie_user_info_list = []

web_control = WebControl()
driver = web_control.getDriver("https://www.megabox.co.kr/movie")

time.sleep(1)

for i in range(10):
    ### 영화목록 1개 추출하기, 클릭해서 정보 페이지로 전환하기
    web_control.click_movie_detail(movie_index=i)
    
    ### 페이지 전환 이후 전환된 페이지를 window(브라우저)에 인식 시키기
    web_control.setWindowPage()
    
    
    ### 영화 상세 정보 추출하기
    title_path = "p.title"
    # 영화제목이 있는 태그 정보 추출하기
    title_element = driver.find_element(By.CSS_SELECTOR, title_path)
    # 추출한 태그에서 영화제목 text 추출하여 출력하기
    title = title_element.text

    ### [개별 관람평점] 추출하기
    use_score_path = "div.story-box > div > div.story-cont > div.story-point > span"
    # - 개별관람평점이 있는 모든 태그 정보 추출하기
    use_score_elements = driver.find_elements(By.CSS_SELECTOR, use_score_path)

    if use_score_elements :
        for i in range(len(use_score_elements)):
            # - 0번째 태그의 텍스트 값 추출하여 출력해보기
            use_score = use_score_elements[i].text
    else :
        use_score = 0

    ### [개별 관람평내용] 추출하기
    # - 개별관람평내용 태그 경로 정의
    use_comment_path = "div.story-area > div.story-box > div > div.story-cont > div.story-txt"
    # - 개별관람평내용 있는 모든 태그 정보 추출하기
    use_comment_elements = driver.find_elements(By.CSS_SELECTOR, use_comment_path)

    if use_comment_elements :
        for c in range(len(use_comment_elements)):
            # - 0번째 태그의 텍스트 값 추출하여 출력해보기
            use_comment = use_comment_elements[c].text

    else:
        use_comment = 0

    ### [긍정/부정] 추출하기
    pos_neg_path = "div.story-box > div > div.story-cont > div.story-point > span"
    # - 개별관람평점이 있는 모든 태그 정보 추출하기
    pos_neg_elements = driver.find_elements(By.CSS_SELECTOR, pos_neg_path)

    positive_count = 0
    negative_count = 0
    # 개별 관람평점이 존재할 경우
    if pos_neg_elements:
        
        for i in range(len(pos_neg_elements)):
            # - 각 태그의 텍스트 값을 숫자로 변환하고 평가
            pos_neg = pos_neg_elements[i].text
            try:
                pos_neg = float(pos_neg)  # 점수를 숫자 형태로 변환
                if pos_neg >= 6:
                    positive_count += 1
                else:
                    negative_count += 1
            except ValueError:
                print(f"정의 값에 문제가 있습니다. {i + 1}: {pos_neg}")
        
        # print(f"긍정: {positive_count}명, 부정: {negative_count}명")
    else:
        print("유저의 평가가 없습니다.")



    
    # print(f"영화제목 : {title}")
    # print(f"개별관람평 평점 : {use_score}")
    # print(f"개별관람평 내용 : {use_comment}")
    
    # # 이전 페이지로 돌아감
    web_control.go_back()

    ### 데이터리스트에 변수 담기
    movie_user_info_list.append({"영화제목":title,
                                "개별관람평점":use_score,
                                "개별관람평내용":use_comment,
                                "긍정/부정":(f"{positive_count}/{negative_count}")})


### 데이터프레임에 저장하기
movie_user_info = pd.DataFrame(movie_user_info_list)

web_control.shutDriver()

In [2]:
# 웹 제어 클래스 인스턴스 생성
web_control = WebControl()

# 크롬 드라이버 실행
driver = web_control.getDriver("https://www.megabox.co.kr/movie")

# 영화 정보 및 관람평 데이터 저장할 리스트
movie_info_list = []
movie_user_info_list = []

# 영화 정보 추출 로직 수정
for i in range(10):  # 영화 목록에서 10개의 영화 정보 추출
    web_control.click_movie_detail(movie_index=i)
    web_control.setWindowPage()

    movie_title = web_control.title("p.title")
    movie_score = web_control.score("div.story-box > div > div.story-cont > div.story-point > span")
    movie_rate, movie_ranking = web_control.temp("#contents > div.movie-detail-page > div.movie-detail-cont > div.info > div.rate > p.cont")
    movie_audience = web_control.audience("div.info > div.audience > p > em")
    movie_use_score = web_control.use_score("div.story-box > div > div.story-cont > div.story-point > span")
    movie_use_comment = web_control.use_comment("div.story-area > div.story-box > div > div.story-cont > div.story-txt")
    positive_count, negative_count = web_control.pos_neg("div.story-box > div > div.story-cont > div.story-point > span")

    # 이전 페이지로 돌아감
    web_control.go_back()

    # 영화 정보 저장
    movie_info_list.append({
        "영화제목": movie_title,
        "실관람평": movie_score,
        "예매순위": movie_ranking,
        "예매율": movie_rate,
        "누적관객수": movie_audience
    })

    # 사용자별 평점이 있는 경우에만 저장 (개별관람평점이 있을 때만 저장)
    if movie_use_score and movie_use_comment:
        for score, comment in zip(movie_use_score, movie_use_comment):
            # 긍정/부정 기준은 6점 이상 긍정, 그 이하 부정
            sentiment = "긍정" if float(score) >= 6 else "부정"
            movie_user_info_list.append({
                "영화제목": movie_title,
                "개별관람평점": score,
                "개별관람평내용": comment,
                "긍정/부정": sentiment
            })

# 데이터프레임에 저장
movie_info_df = pd.DataFrame(movie_info_list)
movie_user_info_df = pd.DataFrame(movie_user_info_list)

# 크롬 드라이버 종료
web_control.shutDriver()


# # 영화 목록 클릭 및 상세 정보 수집
# for i in range(10):  # 영화 목록에서 10개의 영화 정보 추출
#     web_control.click_movie_detail(movie_index=i)
#     web_control.setWindowPage()

#     movie_title = web_control.title("p.title")
#     movie_score = web_control.score("div.story-box > div > div.story-cont > div.story-point > span")
#     movie_rate, movie_ranking = web_control.temp("#contents > div.movie-detail-page > div.movie-detail-cont > div.info > div.rate > p.cont")
#     movie_audience = web_control.audience("div.info > div.audience > p > em")
#     movie_use_score = web_control.use_score("div.story-box > div > div.story-cont > div.story-point > span")
#     movie_use_comment = web_control.use_comment("div.story-area > div.story-box > div > div.story-cont > div.story-txt")
#     positive_count, negative_count = web_control.pos_neg("div.story-box > div > div.story-cont > div.story-point > span")

#     # 이전 페이지로 돌아감
#     web_control.go_back()

#     # 영화 정보 저장
#     movie_info_list.append({
#         "영화제목": movie_title,
#         "실관람평": movie_score,
#         "예매순위": movie_ranking,
#         "예매율": movie_rate,
#         "누적관객수": movie_audience
#     })

#     # 사용자별 평점을 한 줄씩 저장
#     for score, comment in zip(movie_use_score, movie_use_comment):
#         movie_user_info_list.append({
#             "영화제목": movie_title,
#             "개별관람평점": score,
#             "개별관람평내용": comment,
#             "긍정/부정": f"{positive_count}/{negative_count}"
#         })

# # 데이터프레임에 저장
# movie_info_df = pd.DataFrame(movie_info_list)
# movie_user_info_df = pd.DataFrame(movie_user_info_list)

# # 크롬 드라이버 종료
# web_control.shutDriver()


크롬 브라우저를 실행합니다.
영화 상세 페이지로 이동: 영화 1
유저의 평가가 없습니다.
이전페이지로 돌아갑니다.
영화 상세 페이지로 이동: 영화 2
이전페이지로 돌아갑니다.
영화 상세 페이지로 이동: 영화 3
유저의 평가가 없습니다.
이전페이지로 돌아갑니다.
영화 상세 페이지로 이동: 영화 4
유저의 평가가 없습니다.
이전페이지로 돌아갑니다.
영화 상세 페이지로 이동: 영화 5
이전페이지로 돌아갑니다.
영화 상세 페이지로 이동: 영화 6
이전페이지로 돌아갑니다.
영화 상세 페이지로 이동: 영화 7
유저의 평가가 없습니다.
이전페이지로 돌아갑니다.
영화 상세 페이지로 이동: 영화 8
이전페이지로 돌아갑니다.
영화 상세 페이지로 이동: 영화 9
유저의 평가가 없습니다.
이전페이지로 돌아갑니다.
영화 상세 페이지로 이동: 영화 10
이전페이지로 돌아갑니다.


In [3]:
movie_info_df

,영화제목,실관람평,예매순위,예매율,누적관객수
0,미키 17,0,1,48.3,2792
1,첫 번째 키스,6,2,13.6,3329
2,백수아파트,0,3,5.8,711
3,괜찮아 괜찮아 괜찮아!,0,4,4.4,3394
4,캡틴 아메리카: 브레이브 뉴 월드,8,5,4.4,1343688
5,퇴마록,10,6,4.2,131016
6,컴플리트 언노운,0,7,3.2,2774
7,"그 시절, 우리가 좋아했던 소녀",7,8,2.4,79781
8,말할 수 없는 비밀,0,9,1.8,795003
9,히어,8,10,1.4,11668


In [4]:
movie_user_info_df

,영화제목,개별관람평점,개별관람평내용,긍정/부정
0,첫 번째 키스,6,죽지 마세요... ㅈㅂ,긍정
1,첫 번째 키스,10,커플보다는 부부들이 보면 조금 더 와닿을게 많을것 같아요. 근래 본 영화중에 여운이...,긍정
2,첫 번째 키스,8,배우 조합이 좋았어요,긍정
3,첫 번째 키스,7,꽤 볼 만한 영화였습니다,긍정
4,첫 번째 키스,10,아련하네요 ㅜㅠ 호쿠토의 얼굴이 너무 잘했습니다... 앞머리내리고 안경... 아주 ...,긍정
...,...,...,...,...
95,히어,7,즐겁게 관람했습니다,긍정
96,히어,10,재밌게봤어요 ㅋㅋㅋ ㅎㅎ,긍정
97,히어,4,고루한 30년전 작법,부정
98,히어,8,'지금' 은 역사에 역사 역사에 역사를 이어온 나의 자리. 우리가족의 자리...♡,긍정


In [ ]:
web_control.shutDriver()